# Installation

In [1]:
! pip install contextualized-topic-models==2.2.0
! pip install pyldavis
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-12-10 09:56:54--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c0:3470, 2406:da00:ff00::6b17:d1f5, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downl

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd '/content/drive/MyDrive/브런치추천/'

/content/drive/MyDrive/브런치추천


In [4]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from tqdm import tqdm
import pandas as pd

In [5]:
metadata = pd.read_json('processed_data/metadata.json')

In [6]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [7]:
metadata = pd.read_json('processed_data/metadata.json')
metadata['sentence'] = metadata['title'] + metadata['keyword_list'].apply(lambda x: ' '.join(x)) + metadata['sub_title']
custom_tokenizer = CustomTokenizer(Mecab())
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
documents = metadata['sentence'].tolist()

In [8]:
model_name = 'sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens'
model_name = 'jhgan/ko-sroberta-multitask'

In [9]:
# !pip install bertopic

- [참고링크1](https://github.com/ukairia777/bert-topic-models/blob/main/Combined_Topic_Model(korean).ipynb)
- [참고링크2](https://github.com/ukairia777/bert-topic-models/blob/main/Combined_Topic_Model(korean).ipynb)
- [hugging face sentence bert 링크](https://huggingface.co/snunlp/KR-SBERT-V40K-klueNLI-augSTS)

In [10]:
class BerTopic():

  def __init__(self,
              documents,
              max_features = 3000):
    self.documents = documents
    self.max_features = max_features

  def preprocess(self):
    preprocessed_documents = []
    for line in tqdm(self.documents):
      if line and not line.isdecimal():
        preprocessed_documents.append(line)
    self.preprocessed_documents = preprocessed_documents

  def countvec(self):
    self.preprocess()
    custom_tokenizer = CustomTokenizer(Mecab())
    vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=self.max_features)
    self.bow_embeddings = vectorizer.fit_transform(self.preprocessed_documents)
    self.vocab = vectorizer.get_feature_names()
    self.id2token = {k: v for k, v in zip(range(0, len(self.vocab)), self.vocab)}

  def bert_embedding(self,model_name):
    self.contextualized_embeddings = bert_embeddings_from_list(self.preprocessed_documents, model_name)
  
  def topic_model_data_prep(self,
                            model_name,
                            ):
    self.countvec(self.max_features)
    self.bert_embedding(model_name)
    qt = TopicModelDataPreparation()
    self.dataset = qt.load(self.contextualized_embeddings, self.bow_embeddings, self.id2token)

  def fit(self,
          load_model = None,
          contextual_size = 768,
          n_components = 50,
          num_epochs = 10,
          path = 'model/',
          saved = True,
          ):
    try:
      self.vocab
    except:
      self.countvec()
    ctm = CombinedTM(bow_size=len(self.vocab), contextual_size=contextual_size, n_components=n_components, num_epochs=num_epochs)
    self.model = ctm
    if load_model == None:
      self.model.fit(self.dataset)
    else:
      self.model.load(load_model,epoch=num_epochs)
    if saved: self.model.save(path)

# Bertopic model Fitting

In [11]:
model_name = 'sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens'
model_name = 'jhgan/ko-sroberta-multitask'

In [13]:
bertopic = BerTopic(documents = documents)
bertopic.topic_model_data_prep(model_name = model_name, max_features = 3000)

In [ ]:
bertopic.fit(contextual_size = 768, n_components = 10, num_epochs = 20)

In [ ]:
bertopic.fit(contextual_size = 768, n_components = 50, num_epochs = 20)

In [ ]:
bertopic.fit(contextual_size = 768, n_components = 10, num_epochs = 20)

In [ ]:
bertopic.fit(contextual_size = 768, n_components = 10, num_epochs = 20)

# Saved model 

In [11]:
bertopic = BerTopic(documents)

In [12]:
load_model = '/content/drive/MyDrive/브런치추천/model/contextualized_topic_model_nc_10_tpm_0.0_tpv_0.9_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99'
epoch = 19

In [13]:
bertopic.topic_model_data_prep(model_name = model_name)

100%|██████████| 149751/149751 [00:00<00:00, 2261048.62it/s]
/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Batches:   0%|          | 0/749 [00:00<?, ?it/s]

In [14]:
bertopic.fit(load_model=load_model,num_epochs=epoch)

/usr/local/lib/python3.8/dist-packages/contextualized_topic_models/models/ctm.py:495: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  warnings.warn("This is an experimental feature that we has not been fully tested. Refer to the following issue:"
/usr/local/lib/python3.8/dist-packages/contextualized_topic_models/models/ctm.py:470: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  warnings.warn("This is an experimental feature that we has not been fully tested. Refer to the following issue:"


In [15]:
bertopic.model.get_topics(5)

defaultdict(list,
            {0: ['공감', '세이', '고양이', '드로잉', '그림'],
             1: ['분명', '당당', '보단', '한가요', '건가요'],
             2: ['생각', '인생', '행복', '심리', '시간'],
             3: ['영화', '리뷰', '영어', '공부', '음악'],
             4: ['디자인', '마케팅', '브랜드', '데이터', '기획'],
             5: ['여행', '유럽', '일본', '제주', '호텔'],
             6: ['엄마', '육아', '일기', '아이', '아빠'],
             7: ['사랑', '연애', '친구', '이별', '남자'],
             8: ['다이어트', '운동', '건강', '투자', '음식'],
             9: ['직장', '회사', '생활', '퇴사', '취업']})

In [23]:
# topics_predictions = bertopic.model.get_thetas(bertopic.dataset, n_samples=2)

In [ ]:
result = bertopic.model.get_doc_topic_distribution(bertopic.dataset, n_samples=20)

0it [00:00, ?it/s]

In [ ]:
result